In [1]:
# Install the Kaggle library
!pip install kaggle

!pip install pyspark

# Import necessary libraries
import zipfile
import os
from getpass import getpass
from pyspark.sql import SparkSession

# Prompt user for Kaggle credentials
kaggle_username = input("Enter your Kaggle username: ")
kaggle_key = getpass("Enter your Kaggle API key: ")

# Set Kaggle API key using environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_key

# Download the dataset from Kaggle
!kaggle datasets download -d ahmedshahriarsakib/usa-real-estate-dataset

# Unzip the downloaded file
with zipfile.ZipFile("usa-real-estate-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("data")

# Check if the CSV file exists in the directory
data_dir = "data"
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

if not csv_files:
    print("No CSV files found in the directory.")
else:
    # Initialize Spark session
    spark = SparkSession.builder \
        .appName("Load CSV into Spark") \
        .getOrCreate()

    # Load the CSV file into Spark DataFrame
    csv_file_path = os.path.join(data_dir, csv_files[0])
    df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

    # Show the first few rows of the DataFrame
    df.show()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f560fb795c8b243614ef41ebd4058a9338ee926ac4e242159c3e167233d04655
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
Enter your Kaggle username: hilariwaters
Enter your Kaggle API key: ··········
Dataset URL: https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset
License(s): other
 84% 32.0M/38.2M [00:00<00:00, 162MB/s]
100% 38.2M/38.2M [00:00<00:00, 169MB/s]
+-----------+--------+--------+---+----+--------+---------+-------------+-----------+--------+----------+--------------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|         city|      state|zip_code|house_size|prev_sold_date|
+-----------+--------+--------+---+----+--------+---------+-------------+---

In [2]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

import tensorflow as tf

In [3]:
# Rename DataFrame
realtor_df = df

In [4]:
# Dropping nonbeneficial columns 'brokered_by', 'street', 'prev_sold_date'
columns_to_drop = ['brokered_by', 'street', 'prev_sold_date']
realtor_df = realtor_df.drop(*columns_to_drop)
realtor_df.show(10)

+--------+--------+---+----+--------+-------------+-----------+--------+----------+
|  status|   price|bed|bath|acre_lot|         city|      state|zip_code|house_size|
+--------+--------+---+----+--------+-------------+-----------+--------+----------+
|for_sale|105000.0|  3|   2|    0.12|     Adjuntas|Puerto Rico|     601|     920.0|
|for_sale| 80000.0|  4|   2|    0.08|     Adjuntas|Puerto Rico|     601|    1527.0|
|for_sale| 67000.0|  2|   1|    0.15|   Juana Diaz|Puerto Rico|     795|     748.0|
|for_sale|145000.0|  4|   2|     0.1|        Ponce|Puerto Rico|     731|    1800.0|
|for_sale| 65000.0|  6|   2|    0.05|     Mayaguez|Puerto Rico|     680|      NULL|
|for_sale|179000.0|  4|   3|    0.46|San Sebastian|Puerto Rico|     612|    2520.0|
|for_sale| 50000.0|  3|   1|     0.2|       Ciales|Puerto Rico|     639|    2040.0|
|for_sale| 71600.0|  3|   2|    0.08|        Ponce|Puerto Rico|     731|    1050.0|
|for_sale|100000.0|  2|   1|    0.09|        Ponce|Puerto Rico|     730|    

In [5]:
# List of cities to filter by
cities = ['Kansas City', 'Topeka', 'Denver', 'Dallas']

# Dictionary to map cities to their respective states (Kansas City can be in Missouri or Kansas)
city_states = {
    'Kansas City': ['Missouri', 'Kansas'],
    'Topeka': ['Kansas'],
    'Denver': ['Colorado'],
    'Dallas': ['Texas']
}

# Loop through each city and display the first 5 rows for that city in the correct state(s)
for city, states in city_states.items():
    # Filter the DataFrame for the city and check if it's in one of the correct states
    filtered_df = realtor_df.filter(
        (realtor_df['city'] == city) &
        (realtor_df['state'].isin(states)) &
        (realtor_df['bed'] > 1) &
        (realtor_df['bed'] <= 4) &
        (realtor_df['bath'] <= 3)
    )

    # Check if the filtered dataframe is empty
    if not filtered_df.rdd.isEmpty():
        print(f"First 5 rows for {city}, States: {', '.join(states)}:")
        filtered_df.show(5)
    else:
        print(f"No data available for {city} in {', '.join(states)}")

First 5 rows for Kansas City, States: Missouri, Kansas:
+--------+--------+---+----+--------+-----------+--------+--------+----------+
|  status|   price|bed|bath|acre_lot|       city|   state|zip_code|house_size|
+--------+--------+---+----+--------+-----------+--------+--------+----------+
|for_sale|372000.0|  4|   3|    0.19|Kansas City|Missouri|   64158|    2300.0|
|for_sale| 25000.0|  3|   2|    0.35|Kansas City|Missouri|   64133|    1884.0|
|for_sale| 69950.0|  2|   1|    0.19|Kansas City|Missouri|   64129|    1134.0|
|for_sale|109000.0|  2|   3|    0.07|Kansas City|Missouri|   64134|    2062.0|
|for_sale|137500.0|  4|   1|    0.23|Kansas City|Missouri|   64134|     912.0|
+--------+--------+---+----+--------+-----------+--------+--------+----------+
only showing top 5 rows

First 5 rows for Topeka, States: Kansas:
+--------+--------+---+----+--------+------+------+--------+----------+
|  status|   price|bed|bath|acre_lot|  city| state|zip_code|house_size|
+--------+--------+---+

In [6]:
# Loop through each city and display the first 5 rows for that city in the correct state(s)
for city, states in city_states.items():
    # Filter the DataFrame for the city and check if it's in one of the correct states
    filtered_df = realtor_df.filter(
        (realtor_df['city'] == city) &
        (realtor_df['state'].isin(states)) &
        (realtor_df['bed'] > 1) &
        (realtor_df['bed'] <= 4) &
        (realtor_df['bath'] <= 3)
    )

    # Drop rows with NULL or NaN values
    filtered_df = filtered_df.dropna()

    # Check if the filtered dataframe is empty
    if not filtered_df.rdd.isEmpty():
        print(f"First 5 rows for {city}, States: {', '.join(states)}:")
        filtered_df.show(5)
    else:
        print(f"No data available for {city} in {', '.join(states)}")

First 5 rows for Kansas City, States: Missouri, Kansas:
+--------+--------+---+----+--------+-----------+--------+--------+----------+
|  status|   price|bed|bath|acre_lot|       city|   state|zip_code|house_size|
+--------+--------+---+----+--------+-----------+--------+--------+----------+
|for_sale|372000.0|  4|   3|    0.19|Kansas City|Missouri|   64158|    2300.0|
|for_sale| 25000.0|  3|   2|    0.35|Kansas City|Missouri|   64133|    1884.0|
|for_sale| 69950.0|  2|   1|    0.19|Kansas City|Missouri|   64129|    1134.0|
|for_sale|109000.0|  2|   3|    0.07|Kansas City|Missouri|   64134|    2062.0|
|for_sale|137500.0|  4|   1|    0.23|Kansas City|Missouri|   64134|     912.0|
+--------+--------+---+----+--------+-----------+--------+--------+----------+
only showing top 5 rows

First 5 rows for Topeka, States: Kansas:
+--------+--------+---+----+--------+------+------+--------+----------+
|  status|   price|bed|bath|acre_lot|  city| state|zip_code|house_size|
+--------+--------+---+

In [7]:
import pandas as pd
from pyspark.sql import functions as F

In [8]:
# Convert PySpark DataFrame to Pandas DataFrame
df_pandas = df.toPandas()

# List of cities to analyze and their acceptable states
city_states = {
    'Kansas City': ['Missouri', 'Kansas'],
    'Topeka': ['Kansas'],
    'Denver': ['Colorado'],
    'Dallas': ['Texas']
}

# Convert city_states dictionary to a DataFrame for easier filtering
cities_df = pd.DataFrame([(city, state) for city, states in city_states.items() for state in states], columns=['city', 'state'])

# Merge the cities DataFrame with the main DataFrame to filter correctly by city and state
filtered_df = pd.merge(df_pandas, cities_df, on=['city', 'state'])

# Further filter the DataFrame to ensure the number of bedrooms is 5 or less
filtered_df = filtered_df[filtered_df['bed'] <= 5]

# Group by city, state, and number of bedrooms ('bed'), then calculate count and mean price
result = filtered_df.groupby(['city', 'state', 'bed']).agg(
    Houses_Available=('price', 'count')
).reset_index()

# Display the results
print(result)

           city     state  bed  Houses_Available
0        Dallas     Texas  1.0               411
1        Dallas     Texas  2.0              1518
2        Dallas     Texas  3.0              2964
3        Dallas     Texas  4.0              1625
4        Dallas     Texas  5.0               368
5        Denver  Colorado  1.0               276
6        Denver  Colorado  2.0               675
7        Denver  Colorado  3.0               663
8        Denver  Colorado  4.0               454
9        Denver  Colorado  5.0               172
10  Kansas City    Kansas  1.0                 2
11  Kansas City    Kansas  2.0                79
12  Kansas City    Kansas  3.0               149
13  Kansas City    Kansas  4.0                66
14  Kansas City    Kansas  5.0                17
15  Kansas City  Missouri  1.0                67
16  Kansas City  Missouri  2.0               308
17  Kansas City  Missouri  3.0               737
18  Kansas City  Missouri  4.0               609
19  Kansas City  Mis

In [9]:
# Create an empty dictionary to store DataFrames for each city
city_dataframes = {}

# Iterate through the unique cities in the DataFrame
for city_name in cities:
    # Filter the DataFrame for the current city
    city_df = result[result['city'] == city_name]

    # Drop the 'city' column
    city_df = city_df.drop(columns=['city'])

    # Add the filtered DataFrame to the dictionary
    city_dataframes[city_name] = city_df

# Access each DataFrame using the city name as the key
for city_name, city_df in city_dataframes.items():
    print(f"DataFrame for {city_name}:")
    print(city_df)

print(city_df.columns)

DataFrame for Kansas City:
       state  bed  Houses_Available
10    Kansas  1.0                 2
11    Kansas  2.0                79
12    Kansas  3.0               149
13    Kansas  4.0                66
14    Kansas  5.0                17
15  Missouri  1.0                67
16  Missouri  2.0               308
17  Missouri  3.0               737
18  Missouri  4.0               609
19  Missouri  5.0               140
DataFrame for Topeka:
     state  bed  Houses_Available
20  Kansas  1.0                 9
21  Kansas  2.0                92
22  Kansas  3.0               244
23  Kansas  4.0               112
24  Kansas  5.0                30
DataFrame for Denver:
      state  bed  Houses_Available
5  Colorado  1.0               276
6  Colorado  2.0               675
7  Colorado  3.0               663
8  Colorado  4.0               454
9  Colorado  5.0               172
DataFrame for Dallas:
   state  bed  Houses_Available
0  Texas  1.0               411
1  Texas  2.0              1518
2

In [10]:
# Convert city_states dictionary to a PySpark DataFrame
city_states_df = spark.createDataFrame([(city, state) for city, states in city_states.items() for state in states], ['city', 'state'])

# Merge the cities DataFrame with the main DataFrame to filter correctly by city and state
filtered_df = df.join(city_states_df, on=['city', 'state'], how='inner')

# Further filter the DataFrame to ensure the number of bedrooms is 5 or less
filtered_df = filtered_df.filter(filtered_df['bed'] <= 5)

# Group by city, state, and number of bedrooms ('bed'), then calculate count of houses available
result = filtered_df.groupby(['city', 'state', 'bed']).agg(
    F.count('price').alias('Houses_Available')
).orderBy('city', 'bed')

# Display the results
result.show()

# Create an empty dictionary to store DataFrames for each city
city_dataframes = {}

# Iterate through the unique cities in the DataFrame
for city_name in cities:
    # Filter the DataFrame for the current city
    city_df = result.filter(result['city'] == city_name)

    # Drop the 'city' column
    city_df = city_df.drop('city')

    # Add the filtered DataFrame to the dictionary
    city_dataframes[city_name] = city_df

# Access each DataFrame using the city name as the key
for city_name, city_df in city_dataframes.items():
    print(f"DataFrame for {city_name}:")
    city_df.show()

# Group by city and number of bedrooms, then calculate count and mean price
result = filtered_df.groupby(['city', 'bed']).agg(
    F.count('price').alias('Houses_Available'),
    F.mean('price').alias('Average_Price')
).orderBy('city', 'bed')

# Display the results
result.show()

+-----------+--------+---+----------------+
|       city|   state|bed|Houses_Available|
+-----------+--------+---+----------------+
|     Dallas|   Texas|  1|             411|
|     Dallas|   Texas|  2|            1518|
|     Dallas|   Texas|  3|            2964|
|     Dallas|   Texas|  4|            1625|
|     Dallas|   Texas|  5|             368|
|     Denver|Colorado|  1|             276|
|     Denver|Colorado|  2|             675|
|     Denver|Colorado|  3|             663|
|     Denver|Colorado|  4|             454|
|     Denver|Colorado|  5|             172|
|Kansas City|  Kansas|  1|               2|
|Kansas City|Missouri|  1|              67|
|Kansas City|  Kansas|  2|              79|
|Kansas City|Missouri|  2|             308|
|Kansas City|  Kansas|  3|             149|
|Kansas City|Missouri|  3|             737|
|Kansas City|  Kansas|  4|              66|
|Kansas City|Missouri|  4|             609|
|Kansas City|  Kansas|  5|              17|
|Kansas City|Missouri|  5|      

In [11]:
# Remove rows with NaN values
filtered_df = filtered_df.dropna()

# Create a new column 'City_State' combining city and state
filtered_df = filtered_df.withColumn('City_State', F.concat(F.col('city'), F.lit(', '), F.col('state')))

# Create a new column for price per square foot
filtered_df = filtered_df.withColumn('pr_sqr_ft', filtered_df['price'] / filtered_df['house_size'])

# Display data
filtered_df.show()

+------+-----+-----------+--------+--------+---+----+--------+---------+--------+----------+--------------+-------------+------------------+
|  city|state|brokered_by|  status|   price|bed|bath|acre_lot|   street|zip_code|house_size|prev_sold_date|   City_State|         pr_sqr_ft|
+------+-----+-----------+--------+--------+---+----+--------+---------+--------+----------+--------------+-------------+------------------+
|Dallas|Texas|    26204.0|for_sale|245000.0|  3|   2|    0.12| 575184.0|   75051|    1034.0|    2007-02-15|Dallas, Texas| 236.9439071566731|
|Dallas|Texas|   103999.0|for_sale|269000.0|  3|   2|    0.13| 536639.0|   75051|    1529.0|    2013-01-14|Dallas, Texas|175.93198168737737|
|Dallas|Texas|    14121.0|for_sale|239000.0|  4|   2|    0.18|1643384.0|   75232|    1331.0|    1979-01-01|Dallas, Texas| 179.5642374154771|
|Dallas|Texas|   109950.0|for_sale|230000.0|  3|   2|    0.19| 343815.0|   75232|    1334.0|    2022-02-17|Dallas, Texas|172.41379310344828|
|Dallas|Texas

In [12]:
# Filter the DataFrame to show only rows where the state is 'Kansas'
kansas_df = filtered_df.filter(filtered_df['state'] == 'Kansas')

# Display the filtered DataFrame
kansas_df.show()

+-----------+------+-----------+--------+--------+---+----+--------+---------+--------+----------+--------------+-------------------+------------------+
|       city| state|brokered_by|  status|   price|bed|bath|acre_lot|   street|zip_code|house_size|prev_sold_date|         City_State|         pr_sqr_ft|
+-----------+------+-----------+--------+--------+---+----+--------+---------+--------+----------+--------------+-------------------+------------------+
|Kansas City|Kansas|    25306.0|    sold| 79000.0|  3|   1|    0.13|  45551.0|   66748|    1228.0|    2022-03-30|Kansas City, Kansas| 64.33224755700326|
|Kansas City|Kansas|    71242.0|for_sale|297000.0|  3|   3|    0.23| 968836.0|   66106|    2166.0|    2014-08-20|Kansas City, Kansas| 137.1191135734072|
|Kansas City|Kansas|    75009.0|for_sale| 70000.0|  2|   1|    0.25| 871172.0|   66101|    1149.0|    2018-11-20|Kansas City, Kansas| 60.92254134029591|
|Kansas City|Kansas|    44666.0|for_sale|179000.0|  4|   2|    0.41|  35174.0|   6

In [13]:
# Filter rows for each city using the 'City_State' column
topeka_group_df = filtered_df.filter(filtered_df['City_State'] == 'Topeka, Kansas').toPandas()
kcmo_group_df = filtered_df.filter(filtered_df['City_State'] == 'Kansas City, Missouri').toPandas()
kcks_group_df = filtered_df.filter(filtered_df['City_State'] == 'Kansas City, Kansas').toPandas()
dallas_group_df = filtered_df.filter(filtered_df['City_State'] == 'Dallas, Texas').toPandas()
denver_group_df = filtered_df.filter(filtered_df['City_State'] == 'Denver, Colorado').toPandas()

# Now you can work with each pandas DataFrame
# For example:
# print(topeka_group_df)

In [14]:
print(topeka_group_df.columns)

Index(['city', 'state', 'brokered_by', 'status', 'price', 'bed', 'bath',
       'acre_lot', 'street', 'zip_code', 'house_size', 'prev_sold_date',
       'City_State', 'pr_sqr_ft'],
      dtype='object')


USE THIS CODE

In [16]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

# Define dropdown widgets
dropdown = widgets.Dropdown(
    options=['topeka_group_df', 'kcmo_group_df', 'kcks_group_df', 'dallas_group_df', 'denver_group_df'],
    description='Select DataFrame:'
)

zip_code_dropdown = widgets.Dropdown(options=[], description='Select Zip Code:')

# Function to update the zip code dropdown options based on the selected DataFrame
def update_zip_code_dropdown(change):
    selected_df = globals()[dropdown.value]

    if isinstance(selected_df, pd.DataFrame) and 'zip_code' in selected_df.columns:
        zip_code_dropdown.options = list(selected_df['zip_code'].unique())
    else:
        zip_code_dropdown.options = []

# Register the function to handle changes in the DataFrame dropdown value
dropdown.observe(update_zip_code_dropdown, names='value')

# Function to handle changes in the DataFrame dropdown and display results
def handle_dataframe_change(change):
    clear_output(wait=True)

    # Display the dropdown widgets again
    display(dropdown)
    display(zip_code_dropdown)

    selected_df = globals()[dropdown.value]
    zip_code = zip_code_dropdown.value

    if isinstance(selected_df, pd.DataFrame) and zip_code:
        X = selected_df[['house_size', 'bed', 'zip_code']]
        y = selected_df['price']

        column_transformer = ColumnTransformer(
            [('onehot', OneHotEncoder(handle_unknown='ignore'), ['zip_code'])],
            remainder='passthrough'
        )

        pipeline = Pipeline([
            ('preprocessor', column_transformer),
            ('regressor', LinearRegression())
        ])

        # Filter the DataFrame based on the selected zip code
        filtered_df = selected_df[selected_df['zip_code'] == zip_code]

        if len(filtered_df) > 0:
            X = filtered_df[['house_size', 'bed', 'zip_code']]
            y = filtered_df['price']

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            r_squared = r2_score(y_test, y_pred)

            classifier = RandomForestRegressor()
            classifier.fit(X_train, y_train)
            predictions = classifier.predict(X_test)
            y_pred = classifier.predict(X_test)
            mse_rf = mean_squared_error(y_test, y_pred)
            mae_rf = mean_absolute_error(y_test, y_pred)
            r_squared_rf = r2_score(y_test, y_pred)

            gclassifier = GradientBoostingRegressor(n_estimators=500)
            gclassifier.fit(X_train, y_train)
            predictions_gb = gclassifier.predict(X_test)
            y_pred_gb = gclassifier.predict(X_test)
            mse_gb = mean_squared_error(y_test, y_pred_gb)
            mae_gb = mean_absolute_error(y_test, y_pred_gb)
            r_squared_gb = r2_score(y_test, y_pred_gb)


            # Prediction
            # Example prediction for a house with 2000 sqft, 4 rooms, and zip code 'any'
            X_prediction = pd.DataFrame([[1500, 1, zip_code]], columns=['house_size', 'bed', 'zip_code'])
            predicted_price = pipeline.predict(X_prediction)

            # Print the results
            print("Mean Squared Error (Linear Regression):", mse)
            print("Mean Absolute Error (Linear Regression):", mae)
            print("R-squared (Linear Regression):", r_squared)

            print(" " * 50)

            print("Mean Squared Error (Random Forest):", mse_rf)
            print("Mean Absolute Error (Random Forest):", mae_rf)
            print("R-squared (Random Forest):", r_squared_rf)

            print(" " * 50)

            print("Mean Squared Error (Gradient Boosting):", mse_gb)
            print("Mean Absolute Error (Gradient Boosting):", mae_gb)
            print("R-squared (Gradient Boosting):", r_squared_gb)

            print(" " * 50)

            print("Predicted Price (Linear Regression):", predicted_price)

            print("-" * 50)

# Function to handle changes in the zip code dropdown
def handle_zip_code_change(change):
    handle_dataframe_change(None)

# Register the function to handle changes in the DataFrame dropdown value
dropdown.observe(handle_dataframe_change, names='value')

# Register the function to handle changes in the zip code dropdown value
zip_code_dropdown.observe(handle_zip_code_change, names='value')

# Display the dropdown widgets
display(dropdown)
display(zip_code_dropdown)

Dropdown(description='Select DataFrame:', index=4, options=('topeka_group_df', 'kcmo_group_df', 'kcks_group_df…

Dropdown(description='Select Zip Code:', index=4, options=(80204, 80229, 80219, 80221, 80205, 80209, 80216, 80…

Mean Squared Error (Linear Regression): 13444406003.402328
Mean Absolute Error (Linear Regression): 82325.82177743163
R-squared (Linear Regression): 0.7908769751752213
                                                  
Mean Squared Error (Random Forest): 7072486341.570152
Mean Absolute Error (Random Forest): 63451.47023809524
R-squared (Random Forest): 0.8899899529658065
                                                  
Mean Squared Error (Gradient Boosting): 7384257788.003815
Mean Absolute Error (Gradient Boosting): 63711.65518195477
R-squared (Gradient Boosting): 0.8851404573528576
                                                  
Predicted Price (Linear Regression): [633131.80563773]
--------------------------------------------------
